In [1]:

import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, count, year 


spark = SparkSession.builder \
    .appName("LA_Crime_Query_X") \
    .config("spark.executor.instances", "4") \
    .getOrCreate()


DATA_PATH = "s3a://groups-bucket-dblab-905418150721/group36/processed_data"

# Load Clean Data 
df_crime = spark.read.parquet(f"{DATA_PATH}/crime_data_clean.parquet")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
740,application_1765289937462_0733,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

#  Ξεκινάμε το Spark
spark = SparkSession.builder.appName("Query_Notebook").getOrCreate()

#  Ορίζουμε το Path 
DATA_PATH = "s3a://groups-bucket-dblab-905418150721/group36/processed_data"

# 3. Φορτώνουμε τα καθαρά δεδομένα

print("Loading Clean Data...")
df_crime = spark.read.parquet(f"{DATA_PATH}/crime_data_clean.parquet")

# 4. Τυπώνουμε για επιβεβαίωση
print(f"Data Loaded. Rows: {df_crime.count()}")
df_crime.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Loading Clean Data...
Data Loaded. Rows: 3134980
root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: string (nullable = true)
 |-- DATE OCC: string (nullable = true)
 |-- TIME OCC: integer (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true

In [3]:
Query 2 - Ανα έτος, να βρεθούν τα 3 φυλετικά groups με τα περισσότερα θυματα καταγεγραμμένων εγκλημάτων στο LA.
Τα αποτελέσματα να εμφανιστούν με φθίνουσα σειρά αριθμού θυμάτων ανά φυλετικό γκρουπ, να υπολογιστεί και να εμφανιστεί 
επίσης το ποσοστό επί του συνολικού αριθμού θυμάτων ανά περίπτωση. 
Να υλοποιηθεί το query 2 χρησιμοποιώντας dataframe API και SQL APIs, με 4 executors όπως παραπάνω. 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
invalid syntax (<stdin>, line 1)
  File "<stdin>", line 1
    Query 2 - ??? ????, ?? ??????? ?? 3 ???????? groups ?? ?? ??????????? ?????? ??????????????? ?????????? ??? LA.
          ^
SyntaxError: invalid syntax



In [4]:
# φορτώνουμε τα καθορισμένα δεδομένα και τον πίνακα φυλών από εκεί που τα αποθηκεύσαμε (processed_data)
import time
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, year, count, sum as _sum, desc, rank, round

# Αρχικοποίηση Spark
spark = SparkSession.builder.appName("LA_Crime_Query2").getOrCreate()

# Path δεδομένων 

DATA_PATH = "s3a://groups-bucket-dblab-905418150721/group36/processed_data"

print("--- Loading Data ---")
# Φόρτωση Clean Crime Data
df_crime = spark.read.parquet(f"{DATA_PATH}/crime_data_raw.parquet")

#  Φόρτωση Race Codes
df_race_codes = spark.read.parquet(f"{DATA_PATH}/race_codes.parquet")

print(f"Crime Data Rows: {df_crime.count()}")
print("Data loaded successfully.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--- Loading Data ---
Crime Data Rows: 3138128
Data loaded successfully.

In [5]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, to_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from datetime import datetime


spark = SparkSession.builder.appName("Preprocessing_Fix").getOrCreate()
DATA_PATH = "s3a://groups-bucket-dblab-905418150721/group36/processed_data"

print("1. Reading Raw Data...")
df_crime = spark.read.parquet(f"{DATA_PATH}/crime_data_raw.parquet")

print("2. Parsing Dates & Extracting Year (Python Method)...")

# Schema για την επιστροφή της UDF (Ημερομηνία string + Έτος int)
output_schema = StructType([
    StructField("iso_string", StringType(), True),
    StructField("year_val", IntegerType(), True)
])

# Η συνάρτηση που φτιάχνει τη χρονιά
def parse_date_and_year(date_str):
    if not date_str: return (None, None)
    formats = ["%Y %b %d %I:%M:%S %p", "%m/%d/%Y %I:%M:%S %p", "%m/%d/%Y"]
    for fmt in formats:
        try:
            dt = datetime.strptime(date_str, fmt)
            return (dt.strftime("%Y-%m-%d %H:%M:%S"), dt.year)
        except ValueError: continue
    return (None, None)
parse_udf = udf(parse_date_and_year, output_schema)

# Εφαρμογή της λογικής
df_temp = df_crime \
    .withColumn("parsed_occ", parse_udf(col("DATE OCC")))

df_clean = df_temp \
    .withColumn("DATE_OCC", to_timestamp(col("parsed_occ.iso_string"), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("Year", col("parsed_occ.year_val")) \
    .drop("parsed_occ") \
    .withColumn("Vict_Age", col("Vict Age").cast("integer")) \
    .withColumn("LAT", col("LAT").cast("double")) \
    .withColumn("LON", col("LON").cast("double"))

# Φίλτρα (Null Island & Severe Flag)
df_clean = df_clean.filter((col("LAT") != 0) | (col("LON") != 0))
df_clean = df_clean.withColumn("is_severe", col("Crm Cd Desc").contains("AGGRAVATED ASSAULT"))

print("3. Validating Year Column...")
# Έλεγχος ότι η στήλη Year υπάρχει και έχει δεδομένα
df_clean.select("DATE OCC", "Year").show(5)

print("4. SAVING crime_data_clean.parquet ...")

df_clean.write.mode("overwrite").parquet(f"{DATA_PATH}/crime_data_clean.parquet")

print("SUCCESS: crime_data_clean.parquet created successfully!")  

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1. Reading Raw Data...
2. Parsing Dates & Extracting Year (Python Method)...
3. Validating Year Column...
+--------------------+----+
|            DATE OCC|Year|
+--------------------+----+
|2024 Feb 23 12:00...|2024|
|2024 Jan 29 12:00...|2024|
|2024 Jan 07 12:00...|2024|
|2024 Jul 04 12:00...|2024|
|2024 Sep 11 12:00...|2024|
+--------------------+----+
only showing top 5 rows

4. SAVING crime_data_clean.parquet ...
SUCCESS: crime_data_clean.parquet created successfully!

In [6]:

print("--- Loading Prepared Data ---")


df_crime = spark.read.parquet(f"{DATA_PATH}/crime_data_clean.parquet")
df_race_codes = spark.read.parquet(f"{DATA_PATH}/race_codes.parquet")

# Debug: Επιβεβαίωση ότι υπάρχει η στήλη Year
print("Columns in crime dataset:", df_crime.columns)

# Join Logic
actual_col_1 = df_race_codes.columns[0]
actual_col_2 = df_race_codes.columns[1]
df_race_codes_fixed = df_race_codes \
    .withColumnRenamed(actual_col_1, "Code") \
    .withColumnRenamed(actual_col_2, "Description")

df_joined = df_crime.join(
    df_race_codes_fixed, 
    df_crime["Vict Descent"] == df_race_codes_fixed["Code"], 
    "inner"
).select(
    col("Year"),  
    col("Description").alias("Victim_Descent") 
)

df_joined.cache()
print(f"Dataset Loaded. Rows: {df_joined.count()}")
df_joined.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--- Loading Prepared Data ---
Columns in crime dataset: ['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT', 'LON', 'DATE_OCC', 'Year', 'Vict_Age', 'is_severe']
Dataset Loaded. Rows: 2792396
+----+--------------------+
|Year|      Victim_Descent|
+----+--------------------+
|2020|Hispanic/Latin/Me...|
|2020|               Black|
|2020|               Other|
|2020|         Other Asian|
|2020|             Unknown|
+----+--------------------+
only showing top 5 rows

In [7]:
def benchmark_query(name, func):
    # Force garbage collection 
    spark.catalog.clearCache() 
    # Ξανα-κάνουμε cache μόνο το df_joined που μας ενδιαφέρει
    df_joined.cache()
    df_joined.count() 
    
    print(f"Starting execution for: {name}...")
    start_time = time.time()
    
    # Εκτέλεση Query
    result_df = func()
    
    
    # Χρησιμοποιούμε .show() για να δούμε και τα αποτελέσματα όπως ζητάει η εκφώνηση
    result_df.show(5, truncate=False)
    
    end_time = time.time()
    duration = end_time - start_time
    
    print(f"--> Time taken for {name}: {duration:.4f} seconds\n")
    return duration

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
def query2_dataframe():
    #  Group by Year & Descent
    df_grouped = df_joined.groupBy("Year", "Victim_Descent").agg(count("*").alias("count"))
    
    #  Window Functions
    window_total = Window.partitionBy("Year")
    window_rank = Window.partitionBy("Year").orderBy(col("count").desc())
    
    # Calcs
    df_final = df_grouped \
        .withColumn("total_year", _sum("count").over(window_total)) \
        .withColumn("percentage", round((col("count") / col("total_year")) * 100, 2)) \
        .withColumn("rank", rank().over(window_rank)) \
        .filter(col("rank") <= 3) \
        .select("Year", "Victim_Descent", col("count").alias("#"), col("percentage").alias("%")) \
        .orderBy(col("Year").desc(), col("#").desc())
        
    return df_final

time_df = benchmark_query("Query 2 (DataFrame API)", query2_dataframe)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Starting execution for: Query 2 (DataFrame API)...
+----+----------------------+-----+-----+
|Year|Victim_Descent        |#    |%    |
+----+----------------------+-----+-----+
|2025|Hispanic/Latin/Mexican|34   |40.48|
|2025|Unknown               |24   |28.57|
|2025|White                 |13   |15.48|
|2024|Hispanic/Latin/Mexican|28576|29.05|
|2024|White                 |22958|23.34|
+----+----------------------+-----+-----+
only showing top 5 rows

--> Time taken for Query 2 (DataFrame API): 2.2156 seconds

In [9]:
import time
from pyspark.sql import SparkSession

#  Setup Spark (4 executors, 1 core, 2GB)
spark = SparkSession.builder \
    .appName("LA_Crime_Query2_SQL") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

DATA_PATH = "s3a://groups-bucket-dblab-905418150721/group36/processed_data"

# Load Data
print("--- Loading Data for SQL ---")
df_crime = spark.read.parquet(f"{DATA_PATH}/crime_data_clean.parquet")
df_race = spark.read.parquet(f"{DATA_PATH}/race_codes.parquet")

# 3. Fix Race Codes Column Names (Για να μην έχουμε θέμα στην SQL)
# Μετονομάζουμε τις στήλες σε "Code" και "Description" για σιγουριά
cols = df_race.columns
df_race_fixed = df_race.withColumnRenamed(cols[0], "Code").withColumnRenamed(cols[1], "Description")

# 4. Register Temp Views (ΕΔΩ ΕΙΝΑΙ ΤΟ ΜΥΣΤΙΚΟ)
# Τώρα η SQL μπορεί να βλέπει τους πίνακες "crime_table" και "race_table"
df_crime.createOrReplaceTempView("crime_table")
df_race_fixed.createOrReplaceTempView("race_table")

print("Views created: 'crime_table' and 'race_table' are ready for SQL.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--- Loading Data for SQL ---
Views created: 'crime_table' and 'race_table' are ready for SQL.

In [10]:
def query2_sql_implementation():
    # Γράφουμε το SQL Query σε string
    sql_string = """
    WITH JoinedData AS (
        -- 1. Join για να πάρουμε την περιγραφή της φυλής
        SELECT 
            c.Year, 
            r.Description as Victim_Descent
        FROM crime_table c
        JOIN race_table r ON c.`Vict Descent` = r.Code
    ),
    GroupedData AS (
        -- 2. Group by Year & Descent -> Count
        SELECT 
            Year, 
            Victim_Descent, 
            COUNT(*) as count
        FROM JoinedData
        WHERE Year IS NOT NULL
        GROUP BY Year, Victim_Descent
    ),
    WindowedData AS (
        -- 3. Window Functions για Total και Rank
        SELECT 
            Year, 
            Victim_Descent, 
            count,
            SUM(count) OVER (PARTITION BY Year) as total_year,
            RANK() OVER (PARTITION BY Year ORDER BY count DESC) as rn
        FROM GroupedData
    )
    -- 4. Final Select: Top 3 & Percentage Calculation
    SELECT 
        Year, 
        Victim_Descent, 
        count as `#`, 
        ROUND((count / total_year) * 100, 2) as `%`
    FROM WindowedData
    WHERE rn <= 3
    ORDER BY Year DESC, count DESC
    """
    
    # Εκτέλεση του SQL
    return spark.sql(sql_string)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Συνάρτηση μέτρησης (ίδια με πριν)
def benchmark_query(name, func):
    spark.catalog.clearCache()
    # Κάνουμε cache τους αρχικούς πίνακες για να μετρήσουμε μόνο το SQL execution
    spark.table("crime_table").cache()
    spark.table("crime_table").count()
    
    print(f"Starting execution for: {name}...")
    start_time = time.time()
    
    result_df = func()
    result_df.show(10, truncate=False) # Δείχνουμε τις πρώτες 10 γραμμές
    
    end_time = time.time()
    duration = end_time - start_time
    
    print(f"--> Time taken for {name}: {duration:.4f} seconds\n")
    return duration

# Εκτέλεση
time_sql = benchmark_query("Query 2 (SQL API)", query2_sql_implementation)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Starting execution for: Query 2 (SQL API)...
+----+----------------------+-----+-----+
|Year|Victim_Descent        |#    |%    |
+----+----------------------+-----+-----+
|2025|Hispanic/Latin/Mexican|34   |40.48|
|2025|Unknown               |24   |28.57|
|2025|White                 |13   |15.48|
|2024|Hispanic/Latin/Mexican|28576|29.05|
|2024|White                 |22958|23.34|
|2024|Unknown               |19984|20.32|
|2023|Hispanic/Latin/Mexican|69400|34.55|
|2023|White                 |44613|22.21|
|2023|Black                 |30504|15.19|
|2022|Hispanic/Latin/Mexican|73111|35.64|
+----+----------------------+-----+-----+
only showing top 10 rows

--> Time taken for Query 2 (SQL API): 1.6039 seconds